<a href="https://colab.research.google.com/github/narindagenta/DEEP_LEARNING_DQLAB_DIGITALENT/blob/main/find_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import re


# ============================================================
# ACCOUNT → CATEGORY MAPPING (RESMI)
# ============================================================

ACCOUNT_CATEGORY_MAP = {
    # PENDAPATAN
    "Pendapatan sewa kos bulanan": "Pendapatan",
    "Layanan katering bulanan": "Pendapatan",
    "Pendapatan laundry penghuni": "Pendapatan",
    "Pendapatan sewa parkir motor": "Pendapatan",
    "Pendapatan sewa parkir mobil": "Pendapatan",
    "Pendapatan denda keterlambatan bayar": "Pendapatan",
    "Pendapatan sewa ruang usaha warung": "Pendapatan",

    # PENGELUARAN
    "Biaya pemeliharaan bangunan dan kamar kos": "Pengeluaran",
    "Biaya kunjungan dokter karena sakit": "Pengeluaran",
    "Pengeluaran / Pembelian obat": "Pengeluaran",
    "Biaya listrik bulanan": "Pengeluaran",
    "Tagihan / biaya air PAM Bulanan": "Pengeluaran",
    "Biaya bulanan layanan internet & Wi-Fi": "Pengeluaran",
    "Biaya renovasi kecil (cat, perbaikan pintu)": "Pengeluaran",
    "Pembelian perabotan kamar kos (kasur, kipas, meja)": "Pengeluaran",
    "Pajak properti / PBB": "Pengeluaran",

    # TARIF BARU
    "Penyesuaian / Perubahan Biaya Pembangunan Kamar Kos": "Tarif Baru",
    "Penyesuaian / Perubahan Tarif atau Harga Sewa Kos": "Tarif Baru",
}


# ============================================================
# KEYWORD → AKUN MAPPING
# ============================================================

KEYWORD_MAP = {
    # ===== PENDAPATAN =====
    "sewa": "Pendapatan sewa kos bulanan",
    "kos": "Pendapatan sewa kos bulanan",
    "kamar": "Pendapatan sewa kos bulanan",
    "katering": "Layanan katering bulanan",
    "catering": "Layanan katering bulanan",
    "laundry": "Pendapatan laundry penghuni",
    "cuci": "Pendapatan laundry penghuni",
    "parkir": "Pendapatan sewa parkir motor",
    "motor": "Pendapatan sewa parkir motor",
    "mobil": "Pendapatan sewa parkir mobil",
    "denda": "Pendapatan denda keterlambatan bayar",
    "telat": "Pendapatan denda keterlambatan bayar",
    "terlambat": "Pendapatan denda keterlambatan bayar",
    "warung": "Pendapatan sewa ruang usaha warung",
    "ruang usaha": "Pendapatan sewa ruang usaha warung",

    # ===== PENGELUARAN =====
    "pemeliharaan": "Biaya pemeliharaan bangunan dan kamar kos",
    "bangunan": "Biaya pemeliharaan bangunan dan kamar kos",
    "dokter": "Biaya kunjungan dokter karena sakit",
    "berobat": "Biaya kunjungan dokter karena sakit",
    "obat": "Pengeluaran / Pembelian obat",
    "flu": "Pengeluaran / Pembelian obat",
    "listrik": "Biaya listrik bulanan",
    "token": "Biaya listrik bulanan",
    "pam": "Tagihan / biaya air PAM Bulanan",
    "air": "Tagihan / biaya air PAM Bulanan",
    "internet": "Biaya bulanan layanan internet & Wi-Fi",
    "wifi": "Biaya bulanan layanan internet & Wi-Fi",
    "indihome": "Biaya bulanan layanan internet & Wi-Fi",
    "renov": "Biaya renovasi kecil (cat, perbaikan pintu)",
    "renovasi": "Biaya renovasi kecil (cat, perbaikan pintu)",
    "cat": "Biaya renovasi kecil (cat, perbaikan pintu)",
    "pintu": "Biaya renovasi kecil (cat, perbaikan pintu)",
    "perbaikan": "Biaya renovasi kecil (cat, perbaikan pintu)",
    "perabot": "Pembelian perabotan kamar kos (kasur, kipas, meja)",
    "kipas": "Pembelian perabotan kamar kos (kasur, kipas, meja)",
    "kasur": "Pembelian perabotan kamar kos (kasur, kipas, meja)",
    "meja": "Pembelian perabotan kamar kos (kasur, kipas, meja)",
    "pajak": "Pajak properti / PBB",
    "pbb": "Pajak properti / PBB",

    # ===== TARIF BARU =====
    "pembangunan": "Penyesuaian / Perubahan Biaya Pembangunan Kamar Kos",
    "perubahan": "Penyesuaian / Perubahan Tarif atau Harga Sewa Kos",
    "penyesuaian": "Penyesuaian / Perubahan Tarif atau Harga Sewa Kos",
    "kenaikan": "Penyesuaian / Perubahan Tarif atau Harga Sewa Kos",
    "tarif": "Penyesuaian / Perubahan Tarif atau Harga Sewa Kos",
    "harga": "Penyesuaian / Perubahan Tarif atau Harga Sewa Kos",
}


# ============================================================
# CLEAN TEXT & NGRAM SIMILARITY
# ============================================================

def clean_text(t):
    if not isinstance(t, str):
        return ""
    t = re.sub(r'[^a-z0-9\s]', ' ', t.lower())
    return re.sub(r'\s+', ' ', t).strip()


def ngram(tokens, n=3):
    return {tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)}


def similarity_ngram(a, b, n=3):

    ta = clean_text(a).split()
    tb = clean_text(b).split()

    if len(ta) < n or len(tb) < n:
        sa, sb = set(ta), set(tb)
        if not sa or not sb:
            return 0.0
        return len(sa & sb) / len(sa | sb)

    na = ngram(ta, n)
    nb = ngram(tb, n)
    if not na or not nb:
        return 0.0
    return len(na & nb) / len(na | nb)



# ============================================================
# KEYWORD MATCHING
# ============================================================

def keyword_fallback(desc):
    d = desc.lower()
    for key, akun in KEYWORD_MAP.items():
        if key in d:
            return akun
    return None



# ============================================================
# MAIN PROCESS
# ============================================================

def process_ngram_matching(
    master_file="master_akun.xlsx",
    master_sheet="Data",
    trx_file="transaksi.xlsx",
    trx_sheet="Data",
    output_file="transaksi_dengan_kategori.xlsx",
    output_sheet="Data"
):

    df_master = pd.read_excel(master_file, sheet_name=master_sheet)
    df_master["Deskripsi"] = df_master["Deskripsi"].astype(str)
    master_desc_list = df_master["Deskripsi"].tolist()

    df_trx = pd.read_excel(trx_file, sheet_name=trx_sheet)

    # Normalisasi "Nomor"
    if "No" in df_trx.columns:
        df_trx.rename(columns={"No": "Nomor"}, inplace=True)
    elif "NO" in df_trx.columns:
        df_trx.rename(columns={"NO": "Nomor"}, inplace=True)

    akun_list = []
    kategori_list = []
    nominal_list = []
    tarif_list = []

    for _, row in df_trx.iterrows():

        desc = str(row["Deskripsi Transaksi"])
        nominal_asli = row["Nominal"]

        # 1️⃣ Keyword
        akun = keyword_fallback(desc)

        # 2️⃣ N-gram fallback
        if akun is None:
            best_score = 0
            best_match = None
            for m in master_desc_list:
                score = similarity_ngram(desc, m)
                if score > best_score:
                    best_score = score
                    best_match = m
            akun = best_match

        # 3️⃣ Determine category
        kategori = ACCOUNT_CATEGORY_MAP.get(akun, "Tidak Ditemukan")

        # ============================================================
        # PATCH: Only 1 account goes to Tarif Baru
        # ============================================================

        if kategori == "Pendapatan":
            nominal = abs(nominal_asli)
            tarif_baru = 0

        elif kategori == "Pengeluaran":
            nominal = -abs(nominal_asli)
            tarif_baru = 0

        elif akun == "Penyesuaian / Perubahan Biaya Pembangunan Kamar Kos":
            nominal = 0
            tarif_baru = abs(nominal_asli)

        elif kategori == "Tarif Baru":
            nominal = nominal_asli
            tarif_baru = abs(nominal_asli)

        elif kategori == "Tidak Ditemukan":
            nominal = nominal_asli
            tarif_baru = abs(nominal_asli)

        else:
            nominal = nominal_asli
            tarif_baru = 0

        akun_list.append(akun)
        kategori_list.append(kategori)
        nominal_list.append(nominal)
        tarif_list.append(tarif_baru)

    # ============================================================
    # OUTPUT ORDER
    # ============================================================

    df_out = df_trx.copy()
    df_out["Akun"] = akun_list
    df_out["Kategori"] = kategori_list
    df_out["Nominal"] = nominal_list
    df_out["Tarif Baru"] = tarif_list

    df_out = df_out[[
        "Nomor",
        "Deskripsi Transaksi",
        "Akun",
        "Kategori",
        "Nominal",
        "Tarif Baru"
    ]]

    df_out.to_excel(output_file, sheet_name=output_sheet, index=False)
    print("✔ File berhasil dibuat:", output_file)



# ============================================================
# ENTRY POINT
# ============================================================

if __name__ == "__main__":
    process_ngram_matching()


✔ File berhasil dibuat: transaksi_dengan_kategori.xlsx


In [10]:
from google.colab import files

files.download("transaksi_dengan_kategori.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>